# Add a Validation Set

In the lecture, we created a curated dataset with **400,000 training items** and **2,000 test items**, but we did not include a validation (dev) set. This notebook demonstrates how to take Ed Donner’s dataset, [ed-donner/pricer-data](https://huggingface.co/datasets/ed-donner/pricer-data), and add a dev set to it.

> **Note**: This notebook heavily uses snippets from the lectures’ `day2.ipynb` of Week 6.

**Download the Updated Dataset**:  
You can find the resulting dataset here: [antonawinkler/pricer-data](https://huggingface.co/datasets/antonawinkler/pricer-data)

In [ ]:
# imports

# Standard libraries
import os
import random
from itertools import chain
from collections import Counter, defaultdict

# Third-party libraries
from dotenv import load_dotenv
from huggingface_hub import login
from datasets import concatenate_datasets, load_dataset, Dataset, DatasetDict
import matplotlib.pyplot as plt
import numpy as np

# Local modules
from items import Item
from loaders import ItemLoader


In [ ]:
# environment

load_dotenv()
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [ ]:
# Log in to HuggingFace

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

In [ ]:
%matplotlib inline

## Load the Original Dataset

Load the original data from McAuley-Lab/Amazon-Reviews-2023.

In [ ]:
dataset_names = [
    "Automotive",
    "Electronics",
    "Office_Products",
    "Tools_and_Home_Improvement",
     "Cell_Phones_and_Accessories",
     "Toys_and_Games",
     "Appliances",
     "Musical_Instruments",
]

In [ ]:
items = []
for dataset_name in dataset_names:
    loader = ItemLoader(dataset_name)
    items.extend(loader.load())

# Create Balanced Dataset

We apply the balancing algorithm from the course.

In [ ]:
slots = defaultdict(list)
for item in items:
    slots[round(item.price)].append(item)

np.random.seed(42)
random.seed(42)
sample = []
for i in range(1, 1000):
    slot = slots[i]
    if i>=240:
        sample.extend(slot)
    elif len(slot) <= 1200:
        sample.extend(slot)
    else:
        weights = np.array([1 if item.category=='Automotive' else 5 for item in slot])
        weights = weights / np.sum(weights)
        selected_indices = np.random.choice(len(slot), size=1200, replace=False, p=weights)
        selected = [slot[i] for i in selected_indices]
        sample.extend(selected)

print(f"There are {len(sample):,} items in the sample")

The output I get is `There are 408,635 items in the sample`

Since there are 400,000 items in the train set of ed-donner/pricer-data, we can aim for a 98/1/1 split.

## Load Ed Donner's Pricer Data Set

In [ ]:
dataset_ori = load_dataset("ed-donner/pricer-data")
train_ori = dataset_ori['train']
test_ori = dataset_ori['test']

## Observation 1: Order of the Data Has Changed

`dataset_without_devset` should be a subset of `sample`. The order however can be different. Let us check this.

I see different results for the following two cells below, indicating that the order has changed.

In [ ]:
sample[0].prompt

In [ ]:
train_ori[0]['text']

## Observation 2: Duplicate Items

As an further challenge, the dataset shows duplicates with identical scrubbed descriptions. For some of these duplicates the prices are identical too (I see 1774), for others they differ (I see 6747).

> **Note**: Below we use `defaultdict(list)` instead of `set` because it allows to inspect duplicates easily.

In [ ]:
PRICE_IS = "\n\nPrice is $"
def get_key(text, price):
    prefix, price_is, _price_nearest_dollar = text.partition(PRICE_IS)
    return f"{prefix}{price_is}{price}"
def get_key_without_price(text):
    prefix, price_is, _price_nearest_dollar = text.partition(PRICE_IS)
    return f"{prefix}"


In [ ]:
# Identify duplicates by text+price
train_ori_dict = defaultdict(list)
for datapoint in train_ori:
    # Creates a key from the text and price (scrubbed)
    key = get_key(datapoint["text"], datapoint["price"])
    train_ori_dict[key].append(datapoint)

# Number of exact duplicates (same text AND same price)
exact_duplicates = len(train_ori) - len(train_ori_dict)
print(f"There are {exact_duplicates} duplicates with the same description and price.")

# Identify duplicates by text alone (ignoring price)
train_ori_dict_no_price = defaultdict(list)
for datapoint in train_ori:
    key_no_price = get_key_without_price(datapoint["text"])
    train_ori_dict_no_price[key_no_price].append(datapoint)

# Number of duplicates that differ in price but share the same text
different_price_duplicates = len(train_ori_dict) - len(train_ori_dict_no_price)
print(f"In addition, there are {different_price_duplicates} data points where the description is duplicated but the price is different.")

# Total number of duplicates if we consider text alone
overall_duplicates = len(train_ori) - len(train_ori_dict_no_price)
print(f"Overall number of duplicates: {overall_duplicates}")

In [ ]:
test_ori_dict = defaultdict(list)
for datapoint in test_ori:
    key = get_key(datapoint['text'], datapoint['price'])
    test_ori_dict[key].append(datapoint)

In [ ]:
sample_dict = defaultdict(list)
for datapoint in sample:
    key = get_key(datapoint.prompt, datapoint.price)
    sample_dict[key].append(datapoint)

In [ ]:
# Check if all data points in train_ori/test_ori are included in the new sample_dict.
missing = []
count_found = 0

for datapoint in chain(train_ori, test_ori):
    key = get_key(datapoint["text"], datapoint["price"])
    if key not in sample_dict:
        missing.append(datapoint)
    else:
        count_found += 1

print(f"We found {count_found} datapoints in sample_dict.")
print(f"We are missing {len(missing)} datapoints that are not present in sample_dict.")

Expected output of the previous cell
```
We found 402000 datapoints in sample_dict.
We are missing 0 datapoints that are not present in sample_dict.
```

## Add Data Points to the Test and Validation Sets

Since we can match all data points in the original train and test sets from `ed-donner/pricer-data`, we’ll now incorporate any *unused* items from our balanced sample into the test set and create a new validation (dev) set. Our goal is to achieve a **98/1/1** split for train, validation, and test.

In [ ]:
sample_not_used_yet = [datapoint for key in sample_dict.keys() - train_ori_dict.keys() - test_ori_dict.keys() for datapoint in sample_dict[key]]

In [ ]:
# As a santity check, let us visually verify that the distribution of sample_still_available is in line with the complete sample.

# Plot the distribution of prices in sample
def plot_price_distribution(items, name):
    prices = [float(item.price) for item in items]
    plt.figure(figsize=(15, 10))
    plt.title(f"{name} - Avg {sum(prices)/len(prices):.2f} and highest {max(prices):,.2f}\n")
    plt.xlabel('Price ($)')
    plt.ylabel('Count')
    # see https://stackoverflow.com/questions/57026223/how-to-re-scale-the-counts-in-a-matplotlib-histogram
    (counts, bins) = np.histogram(prices, bins=range(0, 1000, 10))
    plt.hist(bins[:-1], color="darkblue", bins=bins, weights=counts/len(prices))
    plt.show()    


def plot_category_distribution(items, name):
    category_counts = Counter()
    for item in items:
        category_counts[item.category]+=1
    categories = sorted(category_counts.keys())
    counts = [category_counts[category] for category in categories]

    # plot a pie chart
    plt.figure(figsize=(12, 10))
    plt.pie(counts, labels=categories, autopct='%1.0f%%', startangle=90)
    
    # Add a circle at the center to create a donut chart (optional)
    centre_circle = plt.Circle((0,0), 0.70, fc='white')
    fig = plt.gcf()
    fig.gca().add_artist(centre_circle)
    plt.title(f'{name} - Categories')
    
    # Equal aspect ratio ensures that pie is drawn as a circle
    plt.axis('equal')  

    plt.show()
plot_price_distribution(sample, 'Complete set')
plot_price_distribution(sample_not_used_yet, 'Not used yet')
plot_category_distribution(sample, 'Complete set')
plot_category_distribution(sample_not_used_yet, 'Not used yet')

In [ ]:
# now add the unused items to the validation and test set
random.seed(42)
random.shuffle(sample_not_used_yet)
validation_items = sample_not_used_yet[:4000]
added_test_items = sample_not_used_yet[4000:]

# create Huggingface dataset
validation_dataset = Dataset.from_dict({"text": [item.prompt for item in validation_items], "price": [item.price for item in validation_items]})
added_test_dataset = Dataset.from_dict({"text": [item.prompt for item in added_test_items], "price": [item.price for item in added_test_items]})

dataset = DatasetDict({
    "train": train_ori,
    "test": concatenate_datasets([test_ori, added_test_dataset]),
    "validation": validation_dataset,
})

print(f"Divided into a training set of {dataset['train'].num_rows:,} items, a validation set of {dataset['validation'].num_rows:,} items, and a test set of {dataset['test'].num_rows:,} items")

In [ ]:
# If you're ready to push to the hub, and fill in the dots with your HF username

HF_USER = ...
DATASET_NAME = f"{HF_USER}/pricer-data"
dataset.push_to_hub(DATASET_NAME, private=True)

## Use of Validation Sets

When you train your model in Week 7.

```python
# load the train and validation set
train = load_dataset(DATASET_NAME, split='train[:100%]') # or less than 100%
validation = load_dataset(DATASET_NAME, split='validation[:100%]') # or less than 100% 

# Define training parameters
train_parameters = SFTConfig(
    eval_strategy="steps", # or "epoch"
    eval_steps=EVAL_STEPS,
    ...
)

# Initialize fine-tuning with validation set
fine_tuning = SFTTrainer(
    eval_dataset=validation,
    ...
)
```

## A Closer Look at the Duplicates

We have now created a dataset that includes a validation set and additional test data. During this process, we observed that **2% of the data contains duplicates**, where the scrubbed descriptions are identical.

Duplicates can contribute to model overfitting. However, since only **2% of the dataset is duplicated**, the impact is likely minimal. Moreover, many of these duplicates actually refer to different physical objects rather than being true duplicates.

### False Duplicates

The “duplicates” we observe are often not duplicates in the original dataset. Minor differences in product descriptions may be removed by the scrubbing process, leading to items that *appear* identical but aren’t. For example:

```
<RinoGear Screen Protector Designed for Sony Xperia XZ Screen Protector Case Friendly Accessories Flexible Full Coverage Clear TPU Film = $0.95>
<RinoGear (2-Pack) Screen Protector Designed for Sony Xperia XZ Screen Protector Case Friendly Accessories Flexible Full Coverage Clear TPU Film = $2.95>
```
"(2-Pack)" is removed in the scrub method.

Similarly:
```
[<EBC Brakes USR7115 USR Series Sport Slotted Rotor = $31.22>,
 <EBC Brakes USR7314 USR Series Sport Slotted Rotor = $71.46>,
 <EBC Brakes USR7409 USR Series Sport Slotted Rotor = $88.67>,
...
 <EBC Brakes USR7305 USR Series Sport Slotted Rotor = $406.55>,
 <EBC Brakes USR7384 USR Series Sport Slotted Rotor = $413.61>,
 <EBC Brakes USR1602 USR Series Sport Slotted Rotor = $615.1>]
```
These all represent different rotor models. 

**Even when both the scrubbed text and the price are identical**, the items may still refer to distinct products. For instance:
```
<5304486359 Refrigerator Door Handles Set Replacement for Frigidaire FFTR1821QW5A Refrigerator - Compatible with 5304486359 White Door Handles - UpStart Components Brand = $17.99>
<5304486359 Refrigerator Door Handles Set Replacement for Frigidaire FFTR1831QP1 Refrigerator - Compatible with 5304486359 White Door Handles - UpStart Components Brand = $17.99>
```

### True Duplicates
Finding *true* duplicates—where the scrubbed text, price, and underlying real-world product match—seems relatively rare. The following items in the **Appliances** set, for instance, likely refer to the same physical product:
```python
{'main_category': 'Tools & Home Improvement',
 'title': 'Whirlpool 8318084 Lid Switch for Washer',
 'average_rating': 4.6,
 'rating_number': 511,
 'features': ['Works with the following models: Whirlpool 1CLBR5432PQ0, Whirlpool 1CLBR5432PQ1, Whirlpool 1CLSQ9549PG0',
  'This products adds a great value',
  'This product is manufactured in United States',
  'Works with the following models: Whirlpool 1CLBR5432PQ0, Whirlpool 1CLBR5432PQ1, Whirlpool 1CLSQ9549PG0',
  'Whirlpool 1CLSQ9549PG1, Whirlpool 1CLSQ9549PW0',
  'Whirlpool 1CLSQ9549PW1, Whirlpool 1CLSR7010PQ0',
  'Whirlpool 1CLSR7010PQ1, Whirlpool 1CLSR7300PQ0',
  'Genuine Replacement Part'],
 'description': ['Product Description',
  'Part Number 8318084 (AP3180933) replaces 1018522, AH886960, EA886960, PS886960., Easy to use and handle. This products adds a great value This product is manufactured in United States.',
  'From the Manufacturer',
  'Whirlpool 8318084 Lid Switch for Washer. Works with the following models: Whirlpool 1CLBR5432PQ0, Whirlpool 1CLBR5432PQ1, Whirlpool 1CLSQ9549PG0, Whirlpool 1CLSQ9549PG1, Whirlpool 1CLSQ9549PW0, Whirlpool 1CLSQ9549PW1, Whirlpool 1CLSR7010PQ0, Whirlpool 1CLSR7010PQ1, Whirlpool 1CLSR7300PQ0. Genuine Replacement Part.'],
 'price': '25.55',
 'images': {'hi_res': [None],
  'large': ['https://m.media-amazon.com/images/I/31QE91zX0mL._AC_.jpg'],
  'thumb': ['https://m.media-amazon.com/images/I/31QE91zX0mL._AC_US75_.jpg'],
  'variant': ['MAIN']},
 'videos': {'title': ["Your Washer Won't Spin?", '8318084 Washer Lid Switch'],
  'url': ['https://www.amazon.com/vdp/09c00a975b4b46198b5703483f424981?ref=dp_vse_rvc_0',
   'https://www.amazon.com/vdp/3c9b3dc3c93444978d542af3fab13c49?ref=dp_vse_rvc_1'],
  'user_id': ['', '']},
 'store': 'Whirlpool',
 'categories': ['Appliances',
  'Parts & Accessories',
  'Washer Parts & Accessories'],
 'details': '{"Manufacturer": "Whirlpool", "Part Number": "8318084", "Item Weight": "1.34 ounces", "Product Dimensions": "3 x 2 x 2 inches", "Item model number": "8318084", "Is Discontinued By Manufacturer": "No", "Item Package Quantity": "1", "Included Components": "Kkk", "Batteries Included?": "No", "Batteries Required?": "No", "Warranty Description": "Kk", "Best Sellers Rank": {"Tools & Home Improvement": 231142, "Washer Parts & Accessories": 1074}, "Date First Available": "August 7, 2008"}',
 'parent_asin': 'B01CT25N26',
 'bought_together': None,
 'subtitle': None,
 'author': None}

{'main_category': 'Tools & Home Improvement',
 'title': 'Whirlpool 8318084 Lid Switch for Washer',
 'average_rating': 4.6,
 'rating_number': 514,
 'features': ['Works with the following models: Whirlpool 1CLBR5432PQ0, Whirlpool 1CLBR5432PQ1, Whirlpool 1CLSQ9549PG0',
  'This products adds a great value',
  'This product is manufactured in United States',
  'Works with the following models: Whirlpool 1CLBR5432PQ0, Whirlpool 1CLBR5432PQ1, Whirlpool 1CLSQ9549PG0',
  'Whirlpool 1CLSQ9549PG1, Whirlpool 1CLSQ9549PW0',
  'Whirlpool 1CLSQ9549PW1, Whirlpool 1CLSR7010PQ0',
  'Whirlpool 1CLSR7010PQ1, Whirlpool 1CLSR7300PQ0',
  'Genuine Replacement Part'],
 'description': ['Product Description',
  'Part Number 8318084 (AP3180933) replaces 1018522, AH886960, EA886960, PS886960., Easy to use and handle. This products adds a great value This product is manufactured in United States.',
  'From the Manufacturer',
  'Whirlpool 8318084 Lid Switch for Washer. Works with the following models: Whirlpool 1CLBR5432PQ0, Whirlpool 1CLBR5432PQ1, Whirlpool 1CLSQ9549PG0, Whirlpool 1CLSQ9549PG1, Whirlpool 1CLSQ9549PW0, Whirlpool 1CLSQ9549PW1, Whirlpool 1CLSR7010PQ0, Whirlpool 1CLSR7010PQ1, Whirlpool 1CLSR7300PQ0. Genuine Replacement Part.'],
 'price': '25.55',
 'images': {'hi_res': [None],
  'large': ['https://m.media-amazon.com/images/I/31QE91zX0mL._AC_.jpg'],
  'thumb': ['https://m.media-amazon.com/images/I/31QE91zX0mL._AC_US75_.jpg'],
  'variant': ['MAIN']},
 'videos': {'title': ['AMI PARTS,Parts Specialist'],
  'url': ['https://www.amazon.com/vdp/09a12ea79b1a4081a18909825437760b?ref=dp_vse_rvc_0'],
  'user_id': ['']},
 'store': 'Whirlpool',
 'categories': ['Appliances',
  'Parts & Accessories',
  'Washer Parts & Accessories'],
 'details': '{"Manufacturer": "Whirlpool", "Part Number": "8318084", "Item Weight": "1.34 ounces", "Product Dimensions": "3 x 2 x 2 inches", "Item model number": "8318084", "Is Discontinued By Manufacturer": "No", "Item Package Quantity": "1", "Included Components": "kkk", "Batteries Included?": "No", "Batteries Required?": "No", "Warranty Description": "kk", "Best Sellers Rank": {"Tools & Home Improvement": 166821, "Washer Parts & Accessories": 684}, "Date First Available": "August 7, 2008"}',
 'parent_asin': 'B0050O1UR8',
 'bought_together': None,
 'subtitle': None,
 'author': None}
```

### Takeaway
2% of the dataset contains duplicates, but most of these represent different physical objects. It does not appear to be worthwhile to remove them from the dataset. In fact it can be better the keep them to have representative data.
